<a href="https://colab.research.google.com/github/SlyFox579/bdt-2023-25962701/blob/main/PBA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

run('pip install --upgrade pip')

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
run('mkdir -p data')
run('gsutil cp gs://bdt-beam/users_v.csv data/')
run('gsutil cp gs://bdt-beam/orders_v_2022.csv data/')

>> pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2

>> pip install --quiet apache-beam
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.5 MB/s eta 0

In [ ]:
import apache_beam as beam

with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/users_v.csv', skip_header_lines=True)
     |beam.Map(lambda x: x.split(','))
     #|beam.Map(lambda x: x[5])
     |beam.Map(lambda x: x[0] + ';' + x[1] + ';' + x[2] + ';' + x[3]
               + ';' + x[4] + ';' + x[5])
     |beam.Map(lambda x: x.split('-'))
     |beam.Map(lambda x: x[0] + ' , ' + x[1] + ' , ' + x[2])
     |beam.Map(lambda x: x.split('/'))
     |beam.Map(lambda x: x[0] + '-' + x[1] + '-' + x[2])
     #|beam.Map(print)
     |beam.io.WriteToText('data/users_vv2.csv')
  )

In [ ]:
with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/orders_v_2022.csv', skip_header_lines=True)
     |beam.Map(lambda x: x.split(';'))
     |beam.Map(lambda x: (x[1],x))
     |beam.combiners.Count.PerKey()
     #|beam.Map(print)
     |beam.io.WriteToText('data/orders_vvv2.csv')
     #|beam.Map(print)
  )

In [ ]:
with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/orders_vvv2.csv-00000-of-00001')
     |beam.Map(lambda x: x.split(')'))
     |beam.Map(lambda x: x[0])
     |beam.Map(lambda x: x.split("'"))
     |beam.Map(lambda x: x[1] + x[2])
     |beam.Map(lambda x: x.split(", "))
     |beam.Map(lambda x: x[0] + ',' + x[1])
     #|beam.Map(print)
     |beam.io.WriteToText('data/orders_vv3.csv')
  )

In [ ]:
from apache_beam import CoGroupByKey, Map
with beam.Pipeline() as p:
    users = (p | "Read orders" >> beam.io.ReadFromText("data/users_vv2.csv-00000-of-00001")
                | beam.Map(lambda x: x.split(';'))
                |beam.Map(lambda x: x[0] + ',' + x[2])
                | beam.Map(lambda x: x.split(','))
             )

    orders = (p |beam.io.ReadFromText("data/orders_vv3.csv-00000-of-00001")
               | beam.Map(lambda x: x.split(','))
            )

    x = ({"users": users, "orders": orders} | CoGroupByKey()
                                            #|beam.Map(print)
                                            |beam.io.WriteToText('data/combined2.csv')
    )




In [ ]:
#average number of orders
with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/combined2.csv-00000-of-00001')
     |beam.Map(lambda x: x.split("'"))
     |beam.Map(lambda x: x[1] + ',' + x[5] + ',' + x[9])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: x[1] + ',' + x[2])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: (x[0],int(x[1])))
     |beam.combiners.Mean.PerKey()
     |beam.Map(print)
  )

('female', 678.5173913043478)
('male', 678.2220381110191)


In [ ]:
#total number of orders

with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/combined2.csv-00000-of-00001')
     |beam.io.WriteToText('data/total.csv')
  )

with beam.Pipeline() as pipe:
  q = (pipe
     |beam.io.ReadFromText('data/total.csv-00000-of-00001')
     |beam.Map(lambda x: x.split("'"))
     |beam.Map(lambda x: x[1] + ',' + x[5] + ',' + x[9])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: x[1] + ',' + x[2])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: (x[0],int(x[1])))
     |beam.CombinePerKey(sum)
     |beam.Map(print)
  )

  r = (pipe
     |"Read total" >> beam.io.ReadFromText('data/combined2.csv-00000-of-00001')
     |beam.Map(lambda x: x.split("'"))
     |beam.Map(lambda x: x[1] + ',' + x[5] + ',' + x[9])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: x[1] + ',' + x[2])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: (x[0],int(x[1])))
     |beam.combiners.Mean.PerKey()
     |"print" >> beam.Map(print)
  )




('female', 678.5173913043478)
('male', 678.2220381110191)
('female', 780295)
('male', 818614)
